In [1]:
#notebook to download the csv of edges and nodes of a given network
import os
import requests 
import time
import string
import networkx as nx
import itertools
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet as wn #importing it
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor
from google.colab import files

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
def extract_text(df):       #extract the text from the tweets and RT
                            #works ONLY on .csv file
  list_strings = []
  for index in range(len(df)):
    if index % 1000 == 0:
      print(str(index)+' / '+str(len(df)))
    text = df.loc[index]['text']                          #if it is nor trucated nor a RT  i take "text"
    string = -1
    if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
        string = df.loc[index,"extended_tweet"]
    if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
        string = df.loc[index,"retweeted_status"]
    if type(string) == str :
        if(re.search('full_text\':(.+?)https',string) != None):     #if I find "full_text"
          s = re.search('full_text\':(.+?)https',string).group(1)
        if(re.search('text\':(.+?)https',string)!= None):
          s = re.search('text\':(.+?)https',string).group(1)
        else: 
          continue
        list_strings.append(s)
        #print(s)         
    else:
      list_strings.append(text)
      #print(text)
      

  return list_strings

In [3]:
# Cleaning, lemmatising and pos tagging tweets

nltk.download('words')
WORDS = set(nltk.corpus.words.words()) #the last two lines serve to download the corpus of standard English language words
nltk.download('stopwords') #downloading stopwords
STOP_WORDS = set(nltk.corpus.stopwords.words("english")) #taking the stop words from English language
nltk.download('wordnet') #downloading wordnet
nltk.download('averaged_perceptron_tagger') #downloading tagger
tag_map = defaultdict(lambda : wn.NOUN) #here we define that wn.NOUN is the default value for the dict
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def lemma_pos_cleaner(tweet):

    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase 
    tweet = re.sub("'"," ",tweet) # remove aphostrophe

    #basically we use pos_tag function on tokens that we get by applying wordpunct tokenization
    #to tweet (it separates all the words and symbols)
    #then we pass the token along with it's wordnet pos value that we get from the tag_map dictionary (noun, adjective, verb or adverb) to the lemma function (the WordNetLemmatizer())
    lemma_function = WordNetLemmatizer()
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) #lemmatize
  

    # francesco: I removed also all 2 letters words and added specific words, words that appears frequently but are discarded because they are not in the english language
    SPECIFIC_WORDS = ['virus', 'coronavirus', 'covid19', 'covid', 'trump', 'hubei', 'beijing', 'xinjiang', 'jinping', 'korea', 'xinhua', 'india', 'taiwan','johnson','singapore', 'africa', 'japanese', 'france', 'asian', 'australia', 'french', 'asia', 'leishenshan', 'british', 'qingdao', 'fauci', 'america',  'california', 'sichuan', 'malaysia', 'huawei','thailand', 'shandong', 'italy', 'philippines', 'germany', 'facebook', 'african', 'shenzhen', 'tokyo', 'russian','uygur', '5g', 'pompeo', 'vietnam', 'australian', 'cambodia', 'zhejiang', 'yunnan', 'guangdong', 'korean', 'iran', 'washington']
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if (w in WORDS or w in SPECIFIC_WORDS) and len(w)>2 and w not in STOP_WORDS ) #remove stop words
   
    return tweet

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
def frequency_dictionary(df):
  unique_words = {}

  for row in df:
    for word in row.split():
      #if the word is encountered for the first time add to dict as key and set its value to 0
      unique_words.setdefault(word,0)
      #increase the value (i.e the count) of the word by 1 every time it is encountered
      unique_words[word] += 1

  return unique_words

In [50]:
# discard less frequent words
def clean_words(cleaned_text, unuseful_words):
  re_cleaned_text = cleaned_text.copy()
  for txt in range(len(re_cleaned_text)):
    if txt % 2000 == 0:
      print(txt, '/',len(re_cleaned_text))
    w = re_cleaned_text[txt].split()
    for word in unuseful_words:
      while word in w:
        w.remove(word)
    re_cleaned_text[txt] = ' '.join(w)
  return re_cleaned_text

In [94]:
period = '_SeptOct2020'  # '', '_JanFeb2020', '_MarchApril2021', '_SeptOct2020'

In [95]:
China = pd.read_csv('/content/China'+period+'.csv')
USA = pd.read_csv('/content/USA'+period+'.csv')
China_USA = pd.read_csv('/content/China&USA'+period+'.csv')

In [96]:
# number of tweets:
print('China: ', len(China))
print('USA: ', len(USA))
print('China&USA: ', len(China_USA))

China:  1291
USA:  7436
China&USA:  8727


In [97]:
text_China = extract_text(China)
text_USA = extract_text(USA)
text_China_USA = extract_text(China_USA)

0 / 1291
1000 / 1291
0 / 7436
1000 / 7436
2000 / 7436
3000 / 7436
4000 / 7436
5000 / 7436
6000 / 7436
7000 / 7436
0 / 8727
1000 / 8727
2000 / 8727
3000 / 8727
4000 / 8727
5000 / 8727
6000 / 8727
7000 / 8727
8000 / 8727


In [98]:
cleaned_text_China = [lemma_pos_cleaner(txt) for txt in text_China]
cleaned_text_USA = [lemma_pos_cleaner(txt) for txt in text_USA]
cleaned_text_China_USA = [lemma_pos_cleaner(txt) for txt in text_China_USA]

print('China:')
print(cleaned_text_China[0:10])
print()
print('USA:')
print(cleaned_text_USA[0:10])
print()
print('China&USA:')
print(cleaned_text_China_USA[0:10])

China:
['daily case coronavirus infection high since mid summer see number expert express concern rapid acceleration covid case', 'presidential election approach voter see country deeply polarize key issue', 'government impose tough new restriction northern city late', 'covid case surpass million week pandemic take toll labor market economy', 'number coronavirus confirm case surpass one million week many health worker', 'number death africa coronavirus surpass mark world health say', 'become first country western report one million confirm covid infection continent grapple resurgence case', 'become first country western reach million confirm covid case accord health ministry', 'president urge vigilance possible second wave covid', 'germany federal minister health test positive coronavirus local medium report']

USA:
['global covid case surpass million coronavirus news follow coverage', 'since start covid pandemic lot misinformation look common coronavirus', 'myth shoe clothes spread co

In [99]:
freq_dict_China = frequency_dictionary(cleaned_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  2233
USA:  4587
China&USA:  4967


In [100]:
# Most frequent words
print('China')
print([key for key in freq_dict_China.keys() if freq_dict_China[key]>100])
print()
print('USA')
print([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>100])
print()
print('China&USA')
print([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>200])

China
['covid', 'case', 'coronavirus', 'say', 'new', 'china', 'report', 'test', 'president', 'health', 'pandemic', 'country', 'trump', 'vaccine', 'watch', 'positive']

USA
['covid', 'coronavirus', 'say', 'trump', 'vaccine', 'new', 'case', 'test', 'president', 'pandemic', 'positive', 'report', 'house', 'people', 'death', 'day', 'health', 'white', 'first', 'state', 'million', 'infection', 'trial', 'week', 'month', 'world', 'one', 'rise', 'country', 'virus', 'take', 'record', 'china', 'get', 'make', 'could', 'may', 'study', 'johnson', 'time', 'hospital', 'restriction', 'year', 'second', 'two', 'global', 'surge', 'see', 'high', 'spread', 'plan', 'government', 'accord', 'mask', 'minister', 'daily', 'hit', 'work', 'outbreak', 'back', 'election', 'city', 'late', 'official', 'amid', 'show', 'help', 'face', 'since', 'campaign', 'risk', 'top', 'use', 'india', 'follow', 'public', 'home', 'die', 'york', 'debate', 'australia', 'due', 'still', 'curb', 'last', 'close', 'need', 'return', 'know', 'pres

# less frequent words: not performed on data with only covid, coronavirus, vaccine keys

In [32]:
# less frequent words:
thr = 2
print('Less frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]<thr]))
print('More frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]>=thr]))
less_frequent_words_China = [key for key in freq_dict_China.keys() if freq_dict_China[key]<thr]
print()
print('Less frequent USA: ', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]<thr]))
print('More frequent USA:', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>=thr]))
less_frequent_words_USA = [key for key in freq_dict_USA.keys() if freq_dict_USA[key]<thr]
print()
print('Less frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<thr]))
print('More frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>=thr]))
less_frequent_words_China_USA = [key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<thr]

Less frequent China:  1132
More frequent China:  1761

Less frequent USA:  820
More frequent USA: 1920

Less frequent China&USA:  1288
More frequent China&USA:  2658


In [ ]:
# discard less frequent words
def clean_words(cleaned_text, unuseful_words):
  re_cleaned_text = cleaned_text.copy()
  for txt in range(len(re_cleaned_text)):
    if txt % 2000 == 0:
      print(txt, '/',len(re_cleaned_text))
    w = re_cleaned_text[txt].split()
    for word in unuseful_words:
      while word in w:
        w.remove(word)
    re_cleaned_text[txt] = ' '.join(w)
  return re_cleaned_text

In [ ]:
cleaned_mostfreq_text_China = clean_words(cleaned_text_China, less_frequent_words_China)
cleaned_mostfreq_text_USA = clean_words(cleaned_text_USA, less_frequent_words_USA)
cleaned_mostfreq_text_China_USA = clean_words(cleaned_text_China_USA, less_frequent_words_China_USA)

0 / 2438
2000 / 2438
0 / 3185
2000 / 3185
0 / 5623
2000 / 5623
4000 / 5623


In [ ]:
freq_dict_China = frequency_dictionary(cleaned_mostfreq_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostfreq_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostfreq_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  497
USA:  576
China&USA:  941


# Build Network

In [101]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         covid    542
1          case    271
2   coronavirus    252
3           say    196
4           new    194
5         china    174
6        report    171
7          test    150
8     president    146
9        health    133
10     pandemic    123
11      country    115
12        trump    114
13      vaccine    114
14        watch    111
15     positive    101
16        world     92
17      million     90
18        death     85
19       people     81
20       accord     80
21          day     72
22       number     67
23         year     64
24    infection     63
25         city     61
26       global     61
27        first     59
28        house     58
29     national     56

USA
           Word  Count
0         covid   4102
1   coronavirus   2144
2           say   1475
3         trump   1264
4       vaccine   1121
5           new   1083
6          case   1012
7          test    987
8     president    974
9      pandemic    575
10     positive    565


In [102]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [72]:
def create_network(cleaned_text):
  network = {}
  #connect the word that appear in the same tweets
  for row in cleaned_text:
    combined_list = [word for word in str.split(row)]
    #for pair in itertools.product(combined_list, combined_list):
    #print(combined_list)
    for pair in itertools.product(combined_list, combined_list):
          #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
          if pair[0]!=pair[1] and not(pair[::-1] in network):
              network.setdefault(pair,0)
              network[pair] += 1 
  network_df = pd.DataFrame.from_dict(network, orient="index")
  network_df.columns = ["weight"]
  network_df.sort_values(by="weight",inplace=True, ascending=False)
  return network, network_df

In [103]:
network_China, network_df_China = create_network(cleaned_text_China)
network_USA, network_df_USA = create_network(cleaned_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_text_China_USA)

In [104]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                     weight
(new, case)             148
(report, case)          145
(case, covid)           141
(china, covid)          112
(test, positive)        108
(covid, new)            105
(report, new)           104
(covid, say)            103
(president, trump)       94
(covid, pandemic)        86
(report, covid)          84
(president, covid)       78
(case, health)           75
(covid, test)            73
(covid, trump)           62
(case, coronavirus)      62
(new, health)            61
(say, president)         59
(white, house)           59
(new, coronavirus)       56
(country, covid)         56
(trump, test)            55
(covid, positive)        55
(covid, million)         54
(case, number)           54
(case, total)            54
(case, million)          54
(death, case)            51
(health, report)         50
(confirm, case)          50

USA:
                          weight
(president, trump)          1010
(say, covid)                 807
(covid, vaccine)    

#Graph


In [75]:
def get_graph(network):
  G = nx.Graph()
  for edge in network:
      G.add_edge(edge[0], edge[1], weight=network[edge])
  return G

In [105]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [106]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  2233
Edges:  46601
Is connected:  False

USA:
Nodes:  4587
Edges:  178578
Is connected:  True

China&USA:
Nodes:  4967
Edges:  207161
Is connected:  True


# PageRank

In [107]:
# Calculating the pagerank on graph G, teleportation probability here is 0.15 but since the graph is strongly connected we can set it to zero if we want
pr_China = nx.algorithms.pagerank(G_China,alpha = 1)
pr_China = dict(sorted(pr_China.items(), key=lambda item: item[1],reverse  = True))

pr_USA = nx.algorithms.pagerank(G_USA,alpha = 1)
pr_USA = dict(sorted(pr_USA.items(), key=lambda item: item[1],reverse  = True))

pr_China_USA = nx.algorithms.pagerank(G_China_USA,alpha = 1)
pr_China_USA = dict(sorted(pr_China_USA.items(), key=lambda item: item[1],reverse  = True))

In [79]:
def threshold(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] >= threshold ]

  return pd.DataFrame(l)

In [80]:
def threshold_reverse(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] < threshold ]

  return pd.DataFrame(l)

In [108]:
thr = 0.0004
print('China: ', len(threshold(pr_China,thr)))
print()
print(threshold(pr_China,thr).iloc[:30])
print()
print('USA: ', len(threshold(pr_USA,thr)))
print()
print(threshold(pr_USA,thr).iloc[:30])
print()
print('China&USA: ', len(threshold(pr_China_USA,thr)))
print()
print(threshold(pr_China_USA,thr).iloc[:30])

China:  518

              0         1
0         covid  0.036656
1          case  0.018083
2   coronavirus  0.016590
3           say  0.015262
4         china  0.014178
5           new  0.013570
6        report  0.012298
7     president  0.010980
8          test  0.010234
9        health  0.009925
10     pandemic  0.008884
11      country  0.008753
12        trump  0.008528
13        world  0.007247
14      vaccine  0.007181
15     positive  0.007175
16      million  0.006152
17          day  0.005901
18       accord  0.005880
19        death  0.005781
20       people  0.005621
21         year  0.004963
22         city  0.004927
23    infection  0.004917
24       number  0.004810
25        first  0.004609
26        house  0.004542
27       global  0.004182
28         hour  0.003947
29     national  0.003926

USA:  513

              0         1
0         covid  0.038157
1   coronavirus  0.023285
2           say  0.018489
3         trump  0.017661
4     president  0.013844
5       vacci

# TF-IDF: not performed

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,1))   # ngram range can be changed to obtain measures regarding n grams instead of single words

X_China = tfidf.fit_transform(cleaned_text_China).toarray()    # entry (i,j) if Tfidf measure of word_list[j] in document i
word_list_China = tfidf.get_feature_names_out()

X_USA = tfidf.fit_transform(cleaned_text_USA).toarray()
word_list_USA = tfidf.get_feature_names_out()

X_China_USA = tfidf.fit_transform(cleaned_text_China_USA).toarray()
word_list_China_USA = tfidf.get_feature_names_out()


In [ ]:
tfidf_df_China = pd.DataFrame(X_China,columns = word_list_China)

tfidf_df_USA = pd.DataFrame(X_USA,columns = word_list_USA)

tfidf_df_China_USA = pd.DataFrame(X_China_USA,columns = word_list_China_USA)

In [ ]:
tfidf_word_measure_China = np.mean(tfidf_df_China,axis = 0)
tfidf_word_measure_China = tfidf_word_measure_China.sort_values(ascending = False)
tfidf_word_measure_USA = np.mean(tfidf_df_USA,axis = 0)
tfidf_word_measure_USA = tfidf_word_measure_USA.sort_values(ascending = False)
tfidf_word_measure_China_USA = np.mean(tfidf_df_China_USA,axis = 0)
tfidf_word_measure_China_USA = tfidf_word_measure_China_USA.sort_values(ascending = False)

In [ ]:
print('China:')
print(tfidf_word_measure_China[0:30])
print()
print('USA:')
print(tfidf_word_measure_USA[0:30])
print()
print('China&USA:')
print(tfidf_word_measure_China_USA[0:30])
print()

China:
covid          0.076069
watch          0.059429
coronavirus    0.047427
case           0.044577
china          0.037314
say            0.037011
new            0.035957
test           0.032966
report         0.031964
president      0.029373
pandemic       0.029153
vaccine        0.026741
trump          0.026549
update         0.026117
health         0.025600
country        0.024406
positive       0.023640
million        0.020711
world          0.020349
people         0.020237
death          0.019359
accord         0.017701
late           0.017458
sept           0.016890
year           0.016416
global         0.016072
day            0.015739
number         0.015450
national       0.014457
infection      0.014375
dtype: float64

USA:
covid          0.063696
coronavirus    0.043593
case           0.032740
say            0.031375
new            0.030973
vaccine        0.029819
trump          0.026958
test           0.026622
president      0.021916
report         0.021543
positive    

# reduced graph

In [109]:
# less important words:
less_important_words_China = list(threshold_reverse(pr_China,thr)[0])

less_important_words_USA = list(threshold_reverse(pr_USA,thr)[0])

less_important_words_China_USA = list(threshold_reverse(pr_China_USA,thr)[0])

In [110]:
cleaned_mostimp_text_China = clean_words(cleaned_text_China,less_important_words_China)
cleaned_mostimp_text_USA = clean_words(cleaned_text_USA,less_important_words_USA)
cleaned_mostimp_text_China_USA = clean_words(cleaned_text_China_USA,less_important_words_China_USA)

0 / 1286
0 / 7431
2000 / 7431
4000 / 7431
6000 / 7431
0 / 8717
2000 / 8717
4000 / 8717
6000 / 8717
8000 / 8717


In [111]:
freq_dict_China = frequency_dictionary(cleaned_mostimp_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostimp_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostimp_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  518
USA:  513
China&USA:  508


In [112]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         covid    542
1          case    271
2   coronavirus    252
3           say    196
4           new    194
5         china    174
6        report    171
7          test    150
8     president    146
9        health    133
10     pandemic    123
11      country    115
12        trump    114
13      vaccine    114
14        watch    111
15     positive    101
16        world     92
17      million     90
18        death     85
19       people     81
20       accord     80
21          day     72
22       number     67
23         year     64
24    infection     63
25         city     61
26       global     61
27        first     59
28        house     58
29     national     56

USA
           Word  Count
0         covid   4102
1   coronavirus   2144
2           say   1475
3         trump   1264
4       vaccine   1121
5           new   1083
6          case   1012
7          test    987
8     president    974
9      pandemic    575
10     positive    565


In [113]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [114]:
network_China, network_df_China = create_network(cleaned_mostimp_text_China)
network_USA, network_df_USA = create_network(cleaned_mostimp_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostimp_text_China_USA)

In [115]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                     weight
(new, case)             148
(report, case)          145
(case, covid)           141
(china, covid)          112
(test, positive)        108
(covid, new)            105
(report, new)           104
(covid, say)            103
(president, trump)       94
(covid, pandemic)        86
(report, covid)          84
(president, covid)       78
(case, health)           75
(covid, test)            73
(case, coronavirus)      62
(covid, trump)           62
(new, health)            61
(white, house)           59
(say, president)         59
(country, covid)         56
(new, coronavirus)       56
(covid, positive)        55
(trump, test)            55
(case, million)          54
(case, total)            54
(covid, million)         54
(case, number)           54
(death, case)            51
(confirm, case)          50
(health, report)         50

USA:
                          weight
(president, trump)          1010
(say, covid)                 807
(covid, vaccine)    

In [116]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [117]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  518
Edges:  19693
Is connected:  False

USA:
Nodes:  513
Edges:  48426
Is connected:  True

China&USA:
Nodes:  508
Edges:  52453
Is connected:  True


#Save edge list

In [118]:
filename = './edgelist_China'+period+'.csv'
nx.write_weighted_edgelist(G_China, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_SeptOct2020.csv
files.download('edgelist_China'+period+'.csv')

filename = './edgelist_USA'+period+'.csv'
nx.write_weighted_edgelist(G_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_USA_SeptOct2020.csv
files.download('edgelist_USA'+period+'.csv')

filename = './edgelist_China_USA'+period+'.csv'
nx.write_weighted_edgelist(G_China_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_USA_SeptOct2020.csv
files.download('edgelist_China_USA'+period+'.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>





# Create Node List




In [61]:
def nodes(freq_dict, name):
  word_nodes = pd.DataFrame.from_dict(freq_dict,orient="index")
  word_nodes.reset_index(inplace=True)
  word_nodes["Label"] = word_nodes["index"]
  word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
  word_nodes = word_nodes.drop(columns=['delete'])
  nodelist = pd.DataFrame()
  nodelist = nodelist.append(word_nodes, ignore_index=True)

  nodelist = nodelist.to_csv("nodelist_"+name+".csv",index=False)
  files.download("nodelist_"+name+".csv")
  return nodelist, word_nodes

In [119]:
nodelist_China, word_nodes_China = nodes(freq_dict_China,'China'+period)
nodelist_USA, word_nodes_USA = nodes(freq_dict_USA,'USA_'+period)
nodelist_China_USA, word_nodes_China_USA = nodes(freq_dict_China_USA,'China_USA'+period)

print('China:')
print(word_nodes_China.head())
print()
print('USA:')
print(word_nodes_USA.head())
print()
print('China&USA:')
print(word_nodes_China_USA.head())
print()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

China:
            Id        Label
0        covid        covid
1         case         case
2  coronavirus  coronavirus
3          say          say
4          new          new

USA:
            Id        Label
0        covid        covid
1  coronavirus  coronavirus
2          say          say
3        trump        trump
4      vaccine      vaccine

China&USA:
            Id        Label
0        covid        covid
1  coronavirus  coronavirus
2          say          say
3        trump        trump
4         case         case

